## Setup Environment

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## Download Model

In [2]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


## Setting Up and Scraping Data from Blog

In [3]:
URL = "https://hackernoon.com/what-good-is-a-metaverse-in-which-you-do-not-truly-own-your-assets"

In [5]:
r = requests.get(URL)

In [8]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'h2', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [9]:
ARTICLE

'What Good is a Metaverse in Which You Do Not Truly Own Your Assets? Entrepreneur. Attorney. Athlete. I write about innovations in blockchain and crypto space. Passionate about DeFi. Non-fungible tokens (NFTs) were one of the hottest things in the blockchain ecosystem in 2021, representing more than $17 billion in trading. This year has been no different as the niche is still finding incredible success as more users, celebrities, and organizations adopt them. When it comes to NFTs - there is no doubt that they will continue to expand into new markets. Some of the big names that have recently joined the craze include Sylvester Stallone, Time Magazine, and Spotify. While this is great news not only for the NFT space but all of the blockchain ecosystem, NFTs are yet to have their true potential unlocked. No matter if you are not into NFTs yourself, chances are you have heard of them mostly in terms of collections. Projects like Bored Ape Yacht Club, Cryptopunks, Axie Infinity, and many ot

## Optimize Data to Work Better with Model

In [10]:
max_chunk = 500

In [11]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [12]:
ARTICLE

'What Good is a Metaverse in Which You Do Not Truly Own Your Assets?<eos> Entrepreneur.<eos> Attorney.<eos> Athlete.<eos> I write about innovations in blockchain and crypto space.<eos> Passionate about DeFi.<eos> Non-fungible tokens (NFTs) were one of the hottest things in the blockchain ecosystem in 2021, representing more than $17 billion in trading.<eos> This year has been no different as the niche is still finding incredible success as more users, celebrities, and organizations adopt them.<eos> When it comes to NFTs - there is no doubt that they will continue to expand into new markets.<eos> Some of the big names that have recently joined the craze include Sylvester Stallone, Time Magazine, and Spotify.<eos> While this is great news not only for the NFT space but all of the blockchain ecosystem, NFTs are yet to have their true potential unlocked.<eos> No matter if you are not into NFTs yourself, chances are you have heard of them mostly in terms of collections.<eos> Projects like B

In [13]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [14]:
len(chunks)

5

## Get Summary

In [15]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [16]:
res[0]

{'summary_text': ' Non-fungible tokens (NFTs) were one of the hottest things in the blockchain ecosystem in 2021, representing more than $17 billion in trading . Some of the big names that have recently joined the craze include Sylvester Stallone, Time Magazine, and Spotify . Gaming has been a great driver in the adoption of technologies in the past, which has also been the case with blockchain technology . While there is a lot of room for NFTs to revolutionize gaming, blockchain technology is still young .'}

In [18]:
summary = ' '.join([summ['summary_text'] for summ in res])

In [19]:
summary

' Non-fungible tokens (NFTs) were one of the hottest things in the blockchain ecosystem in 2021, representing more than $17 billion in trading . Some of the big names that have recently joined the craze include Sylvester Stallone, Time Magazine, and Spotify . Gaming has been a great driver in the adoption of technologies in the past, which has also been the case with blockchain technology . While there is a lot of room for NFTs to revolutionize gaming, blockchain technology is still young .  A Metaverse without NFTs would be one of the hottest topics of 2021 . The Metaverse is all about bringing a life-like experience to its users in terms of liberty, allowing them to interact with it in complex manners . The transparency of the blockchain and its tamper-proof nature ensure no foul play can take place in an industry worth over $10.5 trillion .  Non-fungible tokens (NFTs) were one of the hottest things in the blockchain ecosystem in 2021, representing more than $17 billion in trading . 

## Save Summary as txt File

In [21]:
with open('blogsummary.txt', 'w') as f:
    f.write(summary)